In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import holidays
import pmdarima as pm
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.tsa.statespace.sarimax import SARIMAX
sns.set_theme()

In [2]:
training_data = pd.read_parquet('../Entrenamiento/training_data.parquet')
training_data.head()

,año,mes,dia,hora,dia_semana,holiday,temperature_2m,rain,relative_humidity_2m,snowfall,Bronx,Brooklyn,Manhattan,Queens,Staten Island
datetime,,,,,,,,,,,,,,,
2022-01-01 00:00:00,2022,1,1,0,6,1,8.508500,0.0,97.987221,0.0,4122,8977,16146,5279,266
2022-01-01 01:00:00,2022,1,1,1,6,1,9.308500,0.0,95.387932,0.0,4460,8572,13563,5475,282
2022-01-01 02:00:00,2022,1,1,2,6,1,8.158501,0.0,97.981651,0.0,4253,8785,12271,5293,364
2022-01-01 03:00:00,2022,1,1,3,6,1,8.208500,0.0,99.323456,0.0,3671,8341,11369,4674,352
2022-01-01 04:00:00,2022,1,1,4,6,1,7.808500,0.0,100.000000,0.0,2544,5628,6912,3652,220


In [3]:
def generar_modelos(df, exog=['hora', 'temperature_2m', 'rain', 'relative_humidity_2m', 'snowfall', 'año', 'mes', 'dia', 'dia_semana', 'holiday'],
                    endog=['Bronx', 'Brooklyn', 'Manhattan', 'Queens', 'Staten Island'], train_size = 0.8
                    ):
    n = len(df)
    train_n = int(n * train_size)

    endog_train_data = df[endog][:train_n]
    endog_test_data = df[endog][train_n:]

    exog_train_data = df[exog][:train_n]
    exog_test_data = df[exog][train_n:]

    models = {}
    predictions = {}
    errors = {}

    for district in endog:
        print('Inicio de evaluación de', district)
        model = pm.auto_arima(endog_train_data[district], exogenous=exog_train_data, seasonal=True, m=24)
        order = model.order
        seasonal_order = model.seasonal_order
        print("Order:", order)
        print("Seasonal order:", seasonal_order)

        models[district] = model

        # Obtener las predicciones y los intervalos de confianza
        forecast = model.predict(n_periods=len(endog_test_data), exogenous=exog_test_data, return_conf_int=True)
        predicted_values = forecast[0]
        confidence_intervals = forecast[1]

        # Crear un DataFrame con las predicciones y los intervalos de confianza
        predictions_df = pd.DataFrame({
        'Predicted': predicted_values.values,
        'Lower Confidence Interval': confidence_intervals[:, 0],
        'Upper Confidence Interval': confidence_intervals[:, 1],
        'Real values': endog_test_data[district].values
        })
        predictions_df.set_index(endog_test_data.index, inplace=True)
        predictions[district] = predictions_df

        y_true = endog_test_data[district].values  # valores reales
        y_pred = predictions_df['Predicted'].values  # predicciones del modelo

        mae = mean_absolute_error(y_true, y_pred)
        mse = mean_squared_error(y_true, y_pred)
        rmse = np.sqrt(mse)
        mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100

        errors[district] = {'MAE': mae, 'MSE': mse, 'RMSE': rmse, 'MAPE': mape}
        print('RMSE:', rmse)
        print('-----------------------------------------------------------------------------------------')
    return models, predictions, errors

In [4]:
def graficar_predicciones(predictions):
    for district, data in predictions.items():
        plt.figure(figsize=(20, 10))
        plt.plot(data['Real values'], label='Test Data', color='green')
        plt.plot(data['Predicted'], label='Predictions', color='red')
        plt.fill_between(data.index, data['Lower Confidence Interval'], data['Upper Confidence Interval'], color='pink', alpha=0.3)
        plt.legend()
        plt.title(f'SARIMAX Predictions for {district}')
        plt.xlabel('Date')
        plt.ylabel('Demand')
        plt.show()

In [5]:
models, predictions, errors = generar_modelos(training_data)

Inicio de evaluación de Bronx


MemoryError: Unable to allocate 278. MiB for an array with shape (14016, 51, 51) and data type float64

No puedo entrenar en local el modelo SARIMAX con 2 años de training data